In [3]:
import importlib
import importlib.util
import os
from pathlib import Path
from pprint import pprint as pp

from taccjm import taccjm as tjm

In [4]:
# Dir with this notebook resides
base_dir = Path(os.path.dirname(os.getcwd()))
data_dir = base_dir / "notebooks" / "data"
data_dir.mkdir(exist_ok=True)
base_dir, data_dir

(PosixPath('/Users/carlos/repos/pyadcirc'),
 PosixPath('/Users/carlos/repos/pyadcirc/notebooks/data'))

In [5]:
jm = "l1"
system = "ls6"

# TACCJM Application Development

# Set-Up

Here we go over the basic set-up of how this will be run.

## TACC Job Manager - Initialize


* Lets name our job manager instance `l1`. The name of the JM instance will dictate where all app, job, and other data generated by taccjm will be stored. 

* Choose system to run ADCIRC on - Here we choose ls6 - [Lonestar 6](https://portal.tacc.utexas.edu/user-guides/lonestar5)


In [6]:
jms = tjm.list_jms()
jms

[{'jm_id': 'l1',
  'sys': 'ls6.tacc.utexas.edu',
  'user': 'clos21',
  'apps_dir': '/scratch/06307/clos21/l1/apps',
  'jobs_dir': '/scratch/06307/clos21/l1/jobs'}]

In [7]:
if jm not in [j["jm_id"] for j in jms]:
    tjm.init_jm(jm, system)
else:
    print(f"{jm} already initialized:")
    pp(tjm.get_jm(jm))

l1 already initialized:
{'apps_dir': '/scratch/06307/clos21/l1/apps',
 'jm_id': 'l1',
 'jobs_dir': '/scratch/06307/clos21/l1/jobs',
 'scripts_dir': '/scratch/06307/clos21/l1/scripts',
 'sys': 'ls6.tacc.utexas.edu',
 'trash_dir': '/scratch/06307/clos21/l1/trash',
 'user': 'clos21'}


## Application Operations

In [6]:
tjm.list_apps(jm)

['ls6_adcirc--0.0.0', 'pylauncher--0.0.0']

In [7]:
tjm.get_app(jm, "ls6_adcirc--0.0.0")

{'name': 'ls6_adcirc--0.0.0',
 'short_desc': 'General purpose adcirc application for ls6.',
 'long_desc': '',
 'default_node_count': 1,
 'default_processors_per_node': 10,
 'default_memory_per_node': '1',
 'default_max_run_time': '00:10:00',
 'default_queue': 'development',
 'entry_script': 'run.sh',
 'inputs': [],
 'parameters': [{'name': 'inputDirectory',
   'label': 'Input Directory',
   'desc': 'The directory containing ADCIRC input files.'},
  {'name': 'execDirectory',
   'label': 'Executables Directory',
   'desc': 'The directory containing ADCIRC executables to use.'},
  {'name': 'writeProcesses',
   'label': 'Number of Write Processes',
   'desc': 'Number of processes to dedicate to writing output data.'}],
 'outputs': [{'name': 'Outputs',
   'label': 'Output Directory',
   'desc': 'Directory containing output to archive.'}]}

## Local Applications

In [8]:
for app in (base_dir / "apps").iterdir():
    if not app.name.startswith("."):
        print(f"Application: {app.name}")
        print(f"Assets: {os.listdir(str(app / 'assets'))}")

Application: pylauncher
Assets: ['pylauncher4.py', 'run.sh', 'launch.py', 'hostlist3.py', 'test.sh']
Application: adcirc-remora
Assets: ['.DS_Store', 'test', 'run.sh']
Application: adcirc
Assets: ['.run.sh.swp', '.DS_Store', 'test', 'run.sh']


# ADCIRC App

In [12]:
adcirc_app_path = base_dir / "apps" / "adcirc"
adcirc_job_configs = adcirc_app_path / "job_configs"
adcirc_assets_dir = adcirc_app_path / "assets"

In [18]:
list([f.name for f in adcirc_app_path.iterdir()])

['app-tapis.json',
 '.DS_Store',
 '.ls6.ini.swp',
 'app.json',
 'stampede2.ini',
 'project.ini',
 'frontera.ini',
 'job.json',
 'ls6.ini',
 'assets']

Note how there is an app config for either taccjm or tapis deployments

In [21]:
import json

with open(str(adcirc_app_path / "app.json"), "r") as jf:
    app_config = json.load(jf)
    pp(app_config)

{'default_max_run_time': '00:10:00',
 'default_memory_per_node': '1',
 'default_node_count': 1,
 'default_processors_per_node': 10,
 'default_queue': '{{ app.queue }}',
 'entry_script': 'run.sh',
 'inputs': [],
 'long_desc': '{{ app.long_desc }}',
 'name': '{{ app.name }}--{{ app.version }}',
 'outputs': [{'desc': 'Directory containing output to archive.',
              'label': 'Output Directory',
              'name': 'Outputs'}],
 'parameters': [{'desc': 'The directory containing ADCIRC input files.',
                 'label': 'Input Directory',
                 'name': 'inputDirectory'},
                {'desc': 'The directory containing ADCIRC executables to use.',
                 'label': 'Executables Directory',
                 'name': 'execDirectory'},
                {'desc': 'Number of processes to dedicate to writing output '
                         'data.',
                 'label': 'Number of Write Processes',
                 'name': 'writeProcesses'}],
 'short_desc': 

## Deploying App

In [9]:
adcirc_app_config = {
    "default_max_run_time": "00:10:00",
    "default_memory_per_node": "1",
    "default_node_count": 1,
    "default_processors_per_node": 10,
    "default_queue": "normal",
    "entry_script": "run.sh",
    "inputs": [],
    "long_desc": "",
    "name": "{{ app.name }}--{{ app.version }}",
    "outputs": [
        {
            "desc": "Directory containing output to archive.",
            "label": "Output Directory",
            "name": "Outputs",
        }
    ],
    "parameters": [
        {
            "desc": "The directory containing ADCIRC input files.",
            "label": "Input Directory",
            "name": "inputDirectory",
        },
        {
            "desc": "The directory containing ADCIRC executables to use.",
            "label": "Executables Directory",
            "name": "execDirectory",
        },
        {
            "desc": "Number of processes to dedicate to writing output " "data.",
            "label": "Number of Write Processes",
            "name": "writeProcesses",
        },
        {
            "desc": "Flag indicating whether the REMORA resource monitoring tool should be used",
            "label": "REMORA Flag",
            "name": "remora",
        },
    ],
    "short_desc": "{{ app.short_desc }}",
}

In [ ]:
deployed_app = tjm.deploy_app(
    jm, adcirc_app_config, local_app_dir=str(adcirc_assets_dir)
)

In [17]:
tjm.list_apps(jm)

['ls6_adcirc--0.0.0', 'pylauncher--0.0.0']

In [20]:
tjm.get_app(jm, "pylauncher--0.0.0")

{'name': 'pylauncher--0.0.0',
 'short_desc': 'General purpose pylauncher application',
 'long_desc': ["Wrapper around TACC's pylauncher utility, a paramateric job ",
  'launcher. Takes in a generator.sh shell script and supporting ',
  'files, all in a folder, as input, and alternates between calling ',
  'the generator.sh script to generate an input file for pylauncher ',
  'launching pylauncher with said input file, until the generator.sh ',
  'script generatoes no output for the pylauncher application. For ',
  'further documentation and description see ',
  'https://github.com/cdelcastillo21/tapis-pylauncher.'],
 'default_node_count': 1,
 'default_processors_per_node': 10,
 'default_memory_per_node': '1',
 'default_max_run_time': '00:10:00',
 'default_queue': 'normal',
 'entry_script': 'run.sh',
 'inputs': [{'name': 'job_inputs',
   'label': 'Job Inputs',
   'desc': 'Folder containing at least a generator.sh script and if needed supporting files for jobs to run.'}],
 'parameters': 

# Pylauncher App

In [9]:
pylauncher_app_path = base_dir / "apps" / "pylauncher"
pylauncher_job_configs = pylauncher_app_path / "job_configs"
pylauncher_assets_dir = pylauncher_app_path / "assets"

In [10]:
list([f.name for f in pylauncher_app_path.iterdir()])

['.DS_Store', 'configs', 'assets', 'inputs']

Note how there is an app config for either taccjm or tapis deployments

In [12]:
import json

with open(str(pylauncher_app_path / "app-taccjm.json"), "r") as jf:
    app_config = json.load(jf)
    pp(app_config)

{'default_max_run_time': '00:10:00',
 'default_memory_per_node': '1',
 'default_node_count': 1,
 'default_processors_per_node': 10,
 'default_queue': '{{ app.queue }}',
 'entry_script': 'run.sh',
 'inputs': [{'desc': 'Folder containing at least a generator.sh script and if '
                     'needed supporting files for jobs to run.',
             'label': 'Job Inputs',
             'name': 'job_inputs'}],
 'long_desc': '{{ app.long_desc }}',
 'name': '{{ app.name }}--{{ app.version }}',
 'outputs': [{'desc': 'Directory containing output to archive.',
              'label': 'Output Directory',
              'name': 'outputs'}],
 'parameters': [{'desc': 'Custom tacc modules required for jobs to be execued. '
                         'Will override default.',
                 'label': 'Custom Modules',
                 'name': 'custom_modules'},
                {'default': 'jobs_list.csv',
                 'desc': 'Name of input file for pylauncher (.json or .csv '
                  

## Try a generator out

In [38]:
adcirc_generator_path = pylauncher_job_configs / "adcirc" / "generator.py"

In [48]:
# Use this little trick from importlib to load generator from file at path
spec = importlib.util.spec_from_file_location("generator", str(adcirc_generator_path))
adcirc_gen = importlib.util.module_from_spec(spec)
spec.loader.exec_module(adcirc_gen)
print(adcirc_gen.generator.__doc__)


  Generator for set of basic ADCIRC runs. Assumes base set of files (for
  example, fort.14 adn fort.15 contorl file) are in `base_dir` on TACC systems
  (accessible by compute nodes), and each run's files are in seperate
  directories within `runs_dir`, with the name of the directories being the
  name to give to each run. The same executables are used for each run,
  contained in `execs_dir` and each ADCIRC run is to be executed with a total
  of `cores_per_job` MPIE processes, with `write_proc_per_job` of those being
  dedicated to just writing ADCIRC data output.

  Parameters
  ----------
  base_dir : str
    Dir on TACC where base input files for each ADCIRC run are.
  runs_dir : str
    Dir on TACC containing sub-directories with each runs' job specific files.
  execs_dir : str
    Dir on TACC where ADCIRC executables are.
  cores_per_jobs : int
    Number of total MPI processes to use for each ADCIRC run.
  write_proc_per_job: int, default=0
    Number of teh total cores to us

## Deploying App

In [1]:
app_config = {
    "name": "pylauncher--0.0.1",
    "short_desc": "General purpose pylauncher application",
    "long_desc": (
        "Wrapper around TACC's pylauncher utility, a paramateric job ",
        "launcher. Takes in a generator.sh shell script and supporting ",
        "files, all in a folder, as input, and alternates between calling ",
        "the generator.sh script to generate an input file for pylauncher ",
        "launching pylauncher with said input file, until the generator.sh ",
        "script generatoes no output for the pylauncher application. For ",
        "further documentation and description see ",
        "https://github.com/cdelcastillo21/tapis-pylauncher.",
        "0.1 update : Option to incldue remora flag",
    ),
    "default_node_count": 1,
    "default_processors_per_node": 10,
    "default_memory_per_node": "1",
    "default_max_run_time": "00:10:00",
    "default_queue": "development",
    "entry_script": "run.sh",
    "inputs": [
        {
            "name": "job_inputs",
            "label": "Job Inputs",
            "desc": (
                "Folder containing at least a generator.sh script ",
                "and if needed supporting files for jobs to run.",
            ),
        }
    ],
    "parameters": [
        {
            "name": "custom_modules",
            "label": "Custom Modules",
            "desc": "Custom tacc modules required for jobs to be execued.",
        },
        {
            "name": "pylauncher_input",
            "label": "Pylauncher Input Filename",
            "desc": (
                "Name of pylauncher input file (.json or .csv supported). ",
                "Defaults to jobs_list.csv.",
            ),
            "default": "jobs_list.csv",
        },
        {
            "name": "generator_args",
            "label": "Generator Arguments",
            "desc": "Argument string to pass every call to generator script.",
        },
    ],
    "outputs": [
        {
            "name": "outputs",
            "label": "Output Directory",
            "desc": "Directory containing output to archive.",
        }
    ],
}

In [12]:
deployed_app = tjm.deploy_app(jm, app_config, local_app_dir=str(pylauncher_app_path), proj_config_file=str(pylauncher_app_path / 'configs/taccjm/project.ini'))

In [13]:
tjm.list_apps(jm)

['ls6_adcirc--0.0.0', 'pylauncher--0.0.0', 'pylauncher--0.0.1']

In [14]:
tjm.get_app(jm, "pylauncher--0.0.1")

{'name': 'pylauncher--0.0.1',
 'short_desc': 'General purpose pylauncher application',
 'long_desc': ["Wrapper around TACC's pylauncher utility, a paramateric job ",
  'launcher. Takes in a generator.sh shell script and supporting ',
  'files, all in a folder, as input, and alternates between calling ',
  'the generator.sh script to generate an input file for pylauncher ',
  'launching pylauncher with said input file, until the generator.sh ',
  'script generatoes no output for the pylauncher application. For ',
  'further documentation and description see ',
  'https://github.com/cdelcastillo21/tapis-pylauncher.',
  '0.1 update : Option to incldue remora flag'],
 'default_node_count': 1,
 'default_processors_per_node': 10,
 'default_memory_per_node': '1',
 'default_max_run_time': '00:10:00',
 'default_queue': 'development',
 'entry_script': 'run.sh',
 'inputs': [{'name': 'job_inputs',
   'label': 'Job Inputs',
   'desc': ['Folder containing at least a generator.sh script ',
    'and i

In [ ]:
{
    "name": "{{ app.name }}--{{ app.version }}",
    "short_desc": "{{ app.short_desc }}",
    "long_desc": "{{ app.long_desc }}",
    "default_node_count": "{{ app.nodes }}",
    "default_processors_per_node": "{{ app.tasks_per_node }}",
    "default_memory_per_node": "{{ app.mem_per_node }}",
    "default_max_run_time": "{{ app.runtime }}",
    "default_queue": "{{ app.queue }}",
    "entry_script": "run.sh",
    "inputs": [],
    "parameters": [
        {
            "name": "inputDirectory",
            "label": "Input Directory",
            "desc": "The directory containing ADCIRC input files.",
        },
        {
            "name": "execDirectory",
            "label": "Executables Directory",
            "desc": "The directory containing ADCIRC executables to use.",
        },
        {
            "name": "writeProcesses",
            "label": "Number of Write Processes",
            "desc": "Number of processes to dedicate to writing output data.",
        },
        {
            "name": "remora",
            "label": "REMORA FLag",
            "desc": "Whether to run REMORA resource monitoring tool.",
        },
    ],
    "outputs": [
        {
            "name": "Outputs",
            "label": "Output Directory",
            "desc": "Directory containing output to archive.",
        }
    ],
}